<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/makeSimilarQuestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install peft
!pip install datasets
!pip install bitsandbytes
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 6.9 MB/s eta 0:00:00


In [2]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset

In [3]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
model_path = "meta-llama/Llama-3.1-8B-Instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    # device_map="auto",
    # trust_remote_code=True,
    # use_auth_token=True,
)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [5]:
from sentence_transformers import SentenceTransformer

embedding_model_path = "sentence-transformers/paraphrase-MiniLM-L6-v2"

model_embedding = SentenceTransformer(embedding_model_path, device=device)

model_embedding.eval()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [6]:
dataset = load_dataset("koutch/staqc", 'sca_python')
dataset

0000.parquet:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/85294 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'snippet'],
        num_rows: 85294
    })
})

In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 코사인 유사도를 계산하는 함수
def cosine_similarity(vec1, vec2):
    norm_vec1 = vec1 / np.linalg.norm(vec1)
    norm_vec2 = vec2 / np.linalg.norm(vec2)
    return np.dot(norm_vec1, norm_vec2)

# 두 문장의 코사인 유사도를 계산하는 함수
def get_similarity(sentence1, sentence2):
    # 모델을 통해 임베딩 계산
    embedding1 = model_embedding.encode(sentence1)
    embedding2 = model_embedding.encode(sentence2)

    # 코사인 유사도 계산
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity

# 예시
sentence1 = "What is the capital of France?"
sentence2 = "Which city is the capital of France?"

similarity_score = get_similarity(sentence1, sentence2)
print(f"Cosine Similarity: {similarity_score}")


Cosine Similarity: 0.9393441677093506


In [20]:
model.eval()

from tqdm import tqdm
import csv

csv_file = "generated_similarQuestion.csv"

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

start_row = 0
max_snippet_length = 1024
sliced_dataset = dataset['train'].select(range(5, 7))

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["question_id", "question", "snippet", "similarQuestion", "Confidence"])

    for row in tqdm(sliced_dataset, desc="Generating similar questions", unit="sample"):
        question_id = row['question_id']
        question = row['question']
        snippet = row['snippet']

        if len(snippet) > max_snippet_length:
            snippet = snippet[:max_snippet_length]

        prompt = f"""given a code-related question and the code snippet that answers it, generate a new question that asks about the same code functionality but in a different way.
The new question should have a similar meaning, and both the original and the new question should be answerable with the same code snippet.
Once the new question is generated, indicate completion by generating the token <|eot_id|>.

### Original Question :
{question}
### Code Snippet :
{snippet}
### New Question :
"""
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

        # 입력 토큰의 길이 계산
        input_length = inputs['input_ids'].shape[1]

        # 입력 프롬프트 토큰 수에 128을 더한 max_length 설정
        max_length = input_length + 128

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=max_length,
                eos_token_id=terminators,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.3,  # temperature 값을 조금 높여 다양성 확보
                # top_p=0.9,  # Top-p 샘플링 적용
                # top_k=50,  # Top-k 샘플링 적용
                # use_cache=False
            )

            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            generated_question = ""

            keyword = "### New Question :"
            if keyword in generated_text:
                generated_question = generated_text.split(keyword)[1].strip()

            keyword = "###"
            if keyword in generated_question:
                generated_question = generated_question.split(keyword)[0].strip()

            if not generated_question:
                continue

            similarity = get_similarity(question, generated_question)
            writer.writerow([question_id, question, snippet, generated_question, similarity])



Generating similar questions:  50%|█████     | 1/2 [00:12<00:12, 12.56s/sample]

given a code-related question and the code snippet that answers it, generate a new question that asks about the same code functionality but in a different way.
The new question should have a similar meaning, and both the original and the new question should be answerable with the same code snippet.
Once the new question is generated, indicate completion by generating the token.

### Original Question :
Python: using X and Y values to draw a picture
### Code Snippet :
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from scipy import optimize
%matplotlib inline

# just a gaussian (copy paste from lmfit, another great package)
def my_gaussian(p,x):
    amp = p[0]
    cen = p[1]
    wid = p[2]
    return amp * np.exp(-(x-cen)**2 /wid)

# I do like to write a cost function separately. For the leastsquare algorithm it should return a vector.
def my_cost(p,data):
    return data - my_gaussian(p,data)

# i load the image and generate the x values
image = ndima

Generating similar questions: 100%|██████████| 2/2 [00:25<00:00, 12.55s/sample]

given a code-related question and the code snippet that answers it, generate a new question that asks about the same code functionality but in a different way.
The new question should have a similar meaning, and both the original and the new question should be answerable with the same code snippet.
Once the new question is generated, indicate completion by generating the token.

### Original Question :
How to disable a Tkinter canvas object
### Code Snippet :
from Tkconstants import DISABLED, NORMAL
from Tkinter import Tk, Canvas
import Tkinter
import base64

import ImageTk


_FONTCOLOR = "#FFFFFF"
_BGCOLOR = "#787878"
_ICONDATA = '''iVBORw0KGgoAAAANSUhEUgAAABgAAAAYCAYAAADgdz34AAAAGXRFWHRTb2Z0d2FyZQBBZ
 G9iZSBJbWFnZVJlYWR5ccllPAAAAUJJREFUeNrsVc1qhDAQHq20FnRP7kF68iB9D+99Dp/J5+jdR9Bz8VwU
 PChbQbdo00yYLLuSrW4x9NKBjy+RL5nMZ34MxhjoDBM0h/YExoy3DCYnRtxx3BMbV8QTxxdVvaT7JGYWiV2
 OPbE1GywH9RwDh83xqEiCupHjg6Mmnixa+T5JkrelmuM4fimK4nVJF4bhM6cjLkqWu1vp69P8Q9u2MI7j/P
 OO5hV+yn9wEVEUqRI8yAZOmuc5NE0DWZ

In [19]:
# load generated_similarQuestion.csv
import pandas as pd

df = pd.read_csv('generated_similarQuestion.csv')
# print(df['question'][0])
# print(df['similarQuestion'][0])
# print(df['Confidence'][0])
# print(df)
print(df['snippet'][0])
print(df['snippet'][1])
# print(df['question'][5])
# print(df['question'][6])
# print(df['similarQuestion'][5])
# print(df['similarQuestion'][6])

import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from scipy import optimize
%matplotlib inline

# just a gaussian (copy paste from lmfit, another great package)
def my_gaussian(p,x):
    amp = p[0]
    cen = p[1]
    wid = p[2]
    return amp * np.exp(-(x-cen)**2 /wid)

# I do like to write a cost function separately. For the leastsquare algorithm it should return a vector.
def my_cost(p,data):
    return data - my_gaussian(p,data)

# i load the image and generate the x values
image = ndimage.imread('2d_gaussian.png',flatten=True)
x = np.arange(image.shape[1])
popt = []

# enumerate is a convenient way to loop over an iterable and keep track of the index.
y = []
for index,data in enumerate(image):
    ''' this is the trick to make the algorithm robust. 
    I do plug the index of the maximum value of the current row as
    initial guess for the center. Maybe it would be enough to do
    just that and the fit is unnecessary. Haven`t checked that.
    '''

KeyError: 1

In [27]:
# load generated_similarQuestion.csv
import pandas as pd

df = pd.read_csv('generated_similarQuestion.csv')
# print(df['question'][0])
# print(df['similarQuestion'][0])
# print(df['Confidence'][0])
print(df)
print(df['question'][5])
print(df['question'][6])
print(df['similarQuestion'][5])
print(df['similarQuestion'][6])

   question_id                                           question  \
0     18601725  Using Regex (or simmilar) to change song name ...   
1     21889025                  Product of two beta distributions   
2     26661808           how to grab from JSON in selenium python   
3      7523511  Unable to use wx.NotificationMessage properly ...   
4     34996225          Django: Add additional properties to User   
5     30408713     Python: using X and Y values to draw a picture   
6     31719437             How to disable a Tkinter canvas object   
7     31719440  Fastest way to initialize numpy array with val...   
8       654499                           Python file modes detail   
9     18874387                    Buildbot force build via python   

                                             snippet  \
0  set ASTID to AppleScript's text item delimiter...   
1  set.seed(1) # for reproducability\n\nn <- 1000...   
2  from bs4 import BeautifulSoup\nimport json\n\n...   
3  import wx, sy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!cp generated_similarQuestion.csv /content/drive/MyDrive/


In [ ]:
import pandas as pd

df = pd.read_csv('generated_similarQuestion.csv')
print(df['question'][0])
print(df['similarQuestion'][0])